In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split, KFold

def folder_definition(setup):

    if (os.name == 'nt'):
        if setup==1:
            folder = r"C:\\Users\\dl25365\\works\\madelon\\"
            folder = r"i:\\works\\jupyternotebooks\\madelon\\"
        else:
            folder = r"C:\\Users\\Dror\\Downloads\\madelon\\"
        resultsFolder = folder + "\\results1\\"
        dataFolder = folder + "\\results\\"
        image_path = folder + '\\data\\' 
        folder_mark = '\\'
        augmented_image_path = folder + '\\augmented\\'
    else:
        folder = r"/Users/drorlederman/Documents/works/madelon/"
        resultsFolder = folder + "/results1/"
        dataFolder = folder + "/results/"
        image_path = folder + '/data/' 
        augmented_image_path = folder + '/augmented/' 
        folder_mark = '/'
    return folder, resultsFolder, dataFolder, image_path, augmented_image_path, folder_mark

setup = 0
folder, resultsFolder, dataFolder, image_path, augmented_image_path, folder_mark = folder_definition(setup)
data = pd.read_csv(folder+"madelon_train.data", header=None, delimiter=' ')
labels = pd.read_csv(folder+"madelon_train.labels", header=None)  
data_valid = pd.read_csv(folder+"madelon_valid.data", header=None, delimiter=' ')
labels_valid = pd.read_csv(folder+"madelon_valid.labels", header=None)  
#indices = np.load('indices.npy')
indices = []
data_merged = pd.concat([data, data_valid])
labels_merged = pd.concat([labels, labels_valid])

data = data_merged.copy().reset_index(drop=True)
labels = labels_merged.copy().reset_index(drop=True)
labels[labels == -1] = 0
data = data.dropna(axis=1)

FileNotFoundError: [Errno 2] File b'C:\\\\Users\\\\Dror\\\\Downloads\\\\madelon\\\\madelon_train.data' does not exist: b'C:\\\\Users\\\\Dror\\\\Downloads\\\\madelon\\\\madelon_train.data'

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd
import warnings

sys.path.insert(0, "C:\\Users\\dl25365\\works\\fixedcouponpricer\\fixedcouponpricer")
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold

weights = [1,2,3,4]
no_of_samples = 10000
#cols = list('ABCDEFGHJIKLMNOP')
cols = [str(i) for i in range(0,64)]
data_vect = pd.DataFrame(np.random.randn(no_of_samples, len(cols)), columns=cols)
y = weights[0] * data_vect['0'] + weights[1] * data_vect['1'] + weights[2]*data_vect['2'] + weights[3] * data_vect['3'] 
labels = pd.DataFrame(columns={'dec'})
labels['dec'] = (y > 0)
labels[labels['dec']==True] = 1
labels[labels['dec']==False] = 0
x_train_vect, x_val_vect, x_test_vect, y_train, y_val, y_test = train_val_test_split(data_vect.to_numpy(), labels.to_numpy())
print(x_train_vect.shape, x_val_vect.shape, x_test_vect.shape)

(7000, 64) (1500, 64) (1500, 64)


In [ ]:
batch_size = 256
num_classes = 2
epochs = 50
max_rows = 1000
no_of_batches = 0
no_of_scrambles = 0
augment = False
convert_rgb = True
# input image dimensions
fv = StructDataTransformer()
fv.fit(data_vect.to_numpy(), labels.to_numpy())
img_rows, img_cols = fv.image_dim, fv.image_dim
x_train, x_val, x_test, y_train, y_val, y_test = train_val_test_split(fv.imgs, labels.to_numpy())
fv.augment(x_train, y_train, no_of_batches, no_of_scrambles)
fv.combine_data(x_train, y_train)
x_train = fv.combined_training_imgs.copy()
y_train = fv.combined_training_labels.copy()
input_shape = (img_rows, img_cols, 1)
print(fv.combined_training_imgs.shape, fv.combined_training_labels.shape)
x_train, y_train, x_val, y_val, x_test, y_test, input_shape = pre_process(x_train, y_train, x_val, y_val, x_test, y_test, num_classes)
#pre_process(x_train, y_train, x_val, y_val, x_test, y_test, num_classes):

In [ ]:
model, history = apply_training(x_train, y_train, x_val, y_val, batch_size, epochs, input_shape, num_classes, augment)
score = apply_testing(model, x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

#print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
clf=RandomForestClassifier(n_estimators=1000)
clf.fit(x_train_vect,y_train)
y_pred_val=clf.predict(x_val_vect)
y_pred_test=clf.predict(x_test_vect)
print("Validation accuracy:",metrics.accuracy_score(y_val, y_pred_val))
print("Test set accuracy:",metrics.accuracy_score(y_test, y_pred_test))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Validation accuracy: 0.9393333333333334
Test set accuracy: 0.9473333333333334


In [ ]:
score_val = apply_testing(model, x_val, y_val)
score_test = apply_testing(model, x_test, y_test)
print('Validation loss: ', score_val[0], ' Test loss:', score_test[0])
print('Validation accuracy: ', score_val[1], 'Test accuracy:', score_test[1])

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.base import TransformerMixin
import pandas as pd
import numpy as np
import math
from multiprocessing import Pool
from keras.preprocessing.image import ImageDataGenerator

def train_val_test_split(data_vect, labels, train_size=0.7, val_size=0.15, test_size=0.15):
    x_train_vect, x_test_vect, y_train, y_test = train_test_split(data_vect, labels, test_size=(val_size+test_size))
    x_val_vect, x_test_vect, y_val, y_test = train_test_split(x_test_vect, y_test, test_size=test_size/(1-train_size))
    return x_train_vect, x_val_vect, x_test_vect, y_train, y_val, y_test

def scale(X, x_min, x_max, X_min=[], X_max=[]):
    if (len(X_min) == 0):
        X_min = X.min(axis=0)
    if (len(X_max) == 0):
        X_max = X.max(axis=0)
    nom = (X-X_min)*(x_max-x_min)
    denom = X_max - X_min
    denom[denom==0] = 1
    return (x_min + nom/denom), X_min, X_max

def scramble_image(img):
    no_rows, no_cols, no_of_channels = img.shape[0], img.shape[1], img.shape[2]
    num_pix = no_rows * no_cols * no_of_channels
    img_flat = img.reshape(num_pix, -1)
    perm_vect = np.random.permutation(num_pix)
    return img_flat[perm_vect].reshape(no_rows, no_cols, no_of_channels)

def to_rgb(img):
    return np.repeat(x[..., np.newaxis], 3, -1)    
    
class StructDataTransformer(TransformerMixin):
    """Transforms structured data
    """
    def __init__(self, max_std=None, to_codes=None, ignore=None, randomize_features=False, indices=[], convert_rgb=False):
        self.max_std = max_std
        if not to_codes:
            to_codes = []
        if not ignore:
            ignore = []
        self.ignore_ = ignore
        self.image_dim = 0 
        self.padded_vect = []
        self.img = []
        self.imgs = []
        self.labels = []
        self.augmented_imgs = []
        self.augmented_labels = []
        self.combined_training_imgs = []
        self.combined_training_labels = []
        self.means = []
        self.stds = []
        self.imgs_normalized = []
        self.randomize_features = randomize_features
        self.new_order = []
        self.indices = indices
        self.convert_rgb = convert_rgb
        return
    
    def randomize_features(self, X):
        # function accepts vector
        if self.randomize_features and len(self.new_order) == 0:
            self.new_order = np.random.permutation(X.shape[1])
        return
    
    def permute(self, X, order):
        X = X[:,order]
        return X
    
    def combine_data(self, x_train, y_train):
        if (len(self.augmented_imgs) > 0) & (len(self.augmented_labels) > 0):
            self.combined_training_imgs = np.concatenate((x_train, self.augmented_imgs), axis=0)
            self.combined_training_labels = np.concatenate((y_train, self.augmented_labels), axis=0)
        else:
            self.combined_training_imgs = x_train.copy()
            self.combined_training_labels = y_train.copy()
        return
    
    def augment(self, x_train, y_train, no_of_batches=20, no_of_scrambles=0):
        if (no_of_batches > 0):
            datagen = ImageDataGenerator(
                featurewise_center=True,
                featurewise_std_normalization=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                horizontal_flip=True,
                vertical_flip=True)
            datagen.fit(x_train)        
            train_generator = datagen.flow(x_train, y_train,
                batch_size=batch_size,
                shuffle=False,
                seed=42)
            i = 0
            self.augmented_imgs = []
            self.augmented_labels = []
            for batch in train_generator:
                i += 1
                x_batch, y_batch = train_generator.next()
                x_batch = np.expand_dims(x_batch, axis=0)
                y_batch = np.expand_dims(y_batch, axis=0)
                if (len(self.augmented_imgs) == 0):
                    for row_batch, row_label in zip(x_batch, y_batch):
                        self.augmented_imgs = row_batch
                        self.augmented_labels = row_label
                else:
                    for row_batch, row_label in zip(x_batch, y_batch):
                        self.augmented_imgs = np.append(self.augmented_imgs, row_batch, axis=0) 
                        self.augmented_labels = np.append(self.augmented_labels, row_label, axis=0)
                if i > no_of_batches:
                    break
        if (no_of_scrambles > 0):
            for row, row_label in zip(x_train, y_train):
                for i in range(0, no_of_scrambles):
                    img = scramble_image(row)
                    img = np.expand_dims(img, axis=0)
                    img_label = np.expand_dims(row_label, axis=0)
                    if (len(img_label.shape) == 1):
                        img_label = np.expand_dims(img_label, axis=0)
                    if (len(self.augmented_imgs) == 0):
                        self.augmented_imgs = img
                        self.augmented_labels = img_label
                    else:
                        self.augmented_imgs = np.append(self.augmented_imgs, img, axis=0) 
                        #print(self.augmented_labels.shape, img_label.shape)
                        self.augmented_labels = np.append(self.augmented_labels, img_label, axis=0)
        return self
    
    def calc_padding_order(self, X):
        return (self.image_dim ** 2 - X.shape[1])
    
    def calc_stats(self):
        self.means = self.imgs.mean(axis=(0), dtype='float64')
        self.stds = self.imgs.std(axis=(0), dtype='float64')
        return
    
    def pad(self, X):
        self.image_dim = math.ceil(math.sqrt(X.shape[1]))
        #self.image_dim = 28      # TBD
        no_of_zeros = self.calc_padding_order(X)
        X_padded = np.pad(X, ((0,0),(0, no_of_zeros)), 'edge') # 'constant', constant_values=(0))
        return X_padded
    
    def fit_func(self, X, y):
        img = X.reshape((self.image_dim, self.image_dim, 1), order='F')
        if self.convert_rgb:
            img = to_rgb(img)
        img = np.expand_dims(img, axis=0)
        if (len(self.imgs) == 0):
            self.imgs = img
            self.labels = y
        else:
            self.imgs = np.append(self.imgs, img, axis=0)
            self.labels = np.append(self.labels, y, axis=0)
        return self

    def to_image(self, X, y):
        if self.randomize_features:
            X = self.permute(X, self.new_order)
        if len(self.indices) > 0:
            #print(X[:10,:10])
            X = self.permute(X, self.indices)
            #print(X[:10,:10])
        
        X_padded = self.pad(X)
        for row, row_label in zip(X_padded, y):
            self.fit_func(row, row_label)
        return 
    
    def normalize(self, x_train):
        #self.imgs = (self.imgs - self.means) / (self.stds + 10 ** (-7))
        #self.imgs = (self.imgs - self.means) / (self.stds + 10 ** (-7))
        #X2.mean()

        return scale(x_train, 0, 255)
        
    def fit(self, X, y):
        # pad vector with zeros to get the appropriate length
        #X = scale(X, 0, 255)
        #X /= 255
        self.to_image(X, y)
        
        # calculate statistics
        self.calc_stats()
        
        # normalize
        #self.normalize()
        return

    def transform(self, X, y=None):
        self.to_image(X)
        return 

Using TensorFlow backend.


In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

def pre_process(x_train, y_train, x_val, y_val, x_test, y_test, num_classes):
    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_val = x_val.reshape(x_val.shape[0], 1, img_rows, img_cols)
        x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)
    x_train = x_train.astype('float32')
    x_val = x_val.astype('float32')
    x_test = x_test.astype('float32')
    x_train, X_min, X_max = scale(x_train, 0, 255)
    x_val, _, _ = scale(x_val, 0, 255, X_min=X_min, X_max=X_max)
    x_test, _, _ = scale(x_test, 0, 255, X_min=X_min, X_max=X_max)
    x_train /= 255
    x_val /= 255
    x_test /= 255
    # convert class vectors to binary class matrices
    f = False
    if f:
        i = 0
        for row in x_train:
            x_train[i,:] = preprocess_input(row)
            i = i + 1
        i = 0
        for row in x_val:
            x_val[i,:] = preprocess_input(row)
            i = i + 1
        for row in x_test:
            x_test[i,:] = preprocess_input(row)
            i = i + 1
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_val = keras.utils.to_categorical(y_val, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return x_train, y_train, x_val, y_val, x_test, y_test, input_shape

def apply_training(x_train, y_train, x_val, y_val, batch_size, epochs, input_shape, num_classes, augment=False):
    l2_reg = 0.0001
    l1_reg = 0.0001
    sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    adam = optimizers.Adam(lr=1e-3, epsilon = 1e-8, beta_1 = .9, beta_2 = .999)
    # augmentation
    #datagen = augmentation(x_train, y_train)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(BatchNormalization()) 
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization()) 
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu',
             kernel_regularizer=regularizers.l2(l2_reg),
             activity_regularizer=regularizers.l1(l1_reg)))
    model.add(BatchNormalization()) 
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='sigmoid'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=adam,
                  metrics=['accuracy'])
    #print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, augment)
    #print(input_shape, x_train.shape, y_train.shape, x_test.shape, y_test.shape)
    #if augment:
    #    history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
    #              verbose=1, 
    #              steps_per_epoch=math.ceil(x_train.shape[0]/ batch_size),
    #              validation_data=(x_test, y_test))
    #else:
    history = model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  callbacks=[es],
                  validation_data=(x_val, y_val))
    return model, history

def apply_training_resnet(x_train, y_train, x_test, y_test, batch_size, epochs, input_shape, num_classes, augment=False):
    l2_reg = 0.0001
    l1_reg = 0.0001
    sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    adam = optimizers.Adam(lr=1e-4, epsilon = 1e-8, beta_1 = .9, beta_2 = .999)
    # augmentation
    #datagen = augmentation(x_train, y_train)
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
    
    model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    # add new classifier layers
    flat1 = Flatten()(model.outputs)
    class1 = Dense(1024, activation='relu')(flat1)
    output = Dense(10, activation='softmax')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)

    history = model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=1,
                  callbacks=[es],
                  validation_data=(x_test, y_test))
    return model, history

def apply_testing(model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose=0)
    return score

In [ ]:
batch_size = 128
num_classes = 2
epochs = 100
max_rows = 1000
no_of_batches = 0
no_of_scrambles = 0
augment = False
convert_rgb = True
# input image dimensions
fv = StructDataTransformer(indices=indices, convert_rgb=convert_rgb)
fv.fit(data.to_numpy(), labels.to_numpy())
img_rows, img_cols = fv.image_dim, fv.image_dim
x_train, x_test, y_train, y_test = train_test_split(fv.imgs, labels, test_size=0.33)
fv.augment(x_train, y_train, no_of_batches, no_of_scrambles)
fv.combine_data(x_train, y_train)
print(fv.combined_training_imgs.shape, fv.combined_training_labels.shape)
x_train = fv.combined_training_imgs.copy()
y_train = fv.combined_training_labels.copy()

In [ ]:
input_shape = (img_rows, img_cols, 1)
#x_train, y_train, x_test, y_test, input_shape = pre_process(x_train, y_train, x_test, y_test, num_classes)
score = apply_testing(model)

In [ ]:
model = apply_training(x_train, y_train, x_test, y_test, batch_size, epochs, input_shape, num_classes, augment)
#model = apply_training_resnet(x_train, y_train, x_test, y_test, batch_size, epochs, input_shape, num_classes, augment)


In [ ]:
print(model.summary())

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
import matplotlib.pyplot as plt
augment = False
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train, y_train, x_test, y_test, input_shape = pre_process(x_train, y_train, x_test, y_test)
model, history = apply_training(x_train, y_train, x_test, y_test, batch_size, epochs, input_shape, num_classes, augment)
score = apply_testing(model)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import cv2
import numpy as np

#cv2.imwrite("filename.png", np.zeros((10,10)))
ind = 0
for row in fv.imgs:
#for i in range(fv.imgs.shape[0]):
    if labels.iloc[ind,0] == 0:
        class_folder = folder_mark + '0' + folder_mark
    else:
        class_folder = folder_mark + '1' + folder_mark
    file_name = image_path + class_folder + str(ind) + '.jpg'
    print(file_name)
    cv2.imwrite(file_name, row)
    ind += 1
    
#https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array


img = load_img('bird.jpg')
img.shape

In [ ]:
import numpy as np
import cv2

# Load an color image in grayscale
img = cv2.imread('bird.jpg',0)
no_rows = img.shape[0]
no_cols = img.shape[1]
num_pix = no_rows * no_cols
img_flat = img.reshape(num_pix, -1)
perm_vect = np.random.permutation(num_pix)
img_ar = img_flat[perm_vect]
img_ar2 = img_ar.reshape(no_rows, no_cols)


In [ ]:
img = np.array( [[ 1, 2, 3],
                 [ 4, 2, 5]] )
no_rows = img.shape[0]
no_cols = img.shape[1]
num_pix = no_rows * no_cols
img_flat = img.reshape(num_pix, -1)
perm_vect = np.random.permutation(num_pix)
img_ar = img_flat[perm_vect]
img_ar2 = img_ar.reshape(no_rows, no_cols)
print(img)
print(img_ar2)

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#x_train /= 255
#x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
data.head()

In [ ]:
labels

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
clf=RandomForestClassifier(n_estimators=100)
clf.fit(x_train,y_train)
y_pred=clf.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 2
epochs = 10

# input image dimensions
img_rows, img_cols = 28, 28

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# convert class vectors to binary class matrices

model = Sequential()
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])